In [113]:
from pynq import Overlay
import pynq.lib.dma
from pynq import DefaultIP
from pynq import allocate
from pynq import MMIO

import numpy as np
import pandas as pd
from scipy import stats, signal

from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler #new
from joblib import load

import time
overlay = Overlay("./hardware_ai/cg4002/design_1_wrapper.bit")

In [128]:
# pip install numpy --upgrade --ignore-installed
# !pip install scikit-learn
import warnings

In [63]:
# overlay?

In [64]:
NN_IP_BASE_ADDRESS = 0x80000000
ADDRESS_RANGE = 0x10000
ADDRESS_OFFSET = 0x0

mmio = MMIO(NN_IP_BASE_ADDRESS, ADDRESS_RANGE)
ap_start = 0x1
ap_auto_reset = 0x80
mmio.write(ADDRESS_OFFSET, ap_start|ap_auto_reset)

In [65]:
def compute_mean(data):
    return np.mean(data)

def compute_variance(data):
    return np.var(data)

def compute_median_absolute_deviation(data):
    return stats.median_absolute_deviation(data)

def compute_root_mean_square(data):
    def compose(*fs):
        def wrapped(x):
            for f in fs[::-1]:
                x = f(x)
            return x
        return wrapped
    rms = compose(np.sqrt, np.mean, np.square)
    return rms(data)

def compute_interquartile_range(data):
    return stats.iqr(data)

def compute_percentile_75(data):
    return np.percentile(data, 75)

def compute_kurtosis(data):
    return stats.kurtosis(data)

def compute_min_max(data):
    return np.max(data) - np.min(data)

def compute_signal_magnitude_area(data):
    return np.sum(data) / len(data)

def compute_zero_crossing_rate(data):
    return ((data[:-1] * data[1:]) < 0).sum()

def compute_spectral_centroid(data):
    spectrum = np.abs(np.fft.rfft(data))
    normalized_spectrum = spectrum / np.sum(spectrum)  
    normalized_frequencies = np.linspace(0, 1, len(spectrum))
    spectral_centroid = np.sum(normalized_frequencies * normalized_spectrum)
    return spectral_centroid

def compute_spectral_entropy(data):
    freqs, power_density = signal.welch(data)
    return stats.entropy(power_density)

def compute_spectral_energy(data):
    freqs, power_density = signal.welch(data)
    return np.sum(np.square(power_density))

def compute_principle_frequency(data):
    freqs, power_density = signal.welch(data)
    return freqs[np.argmax(np.square(power_density))]

def extract_raw_data_features_per_row(f_n):
    f1_mean = compute_mean(f_n)
    f1_var = compute_variance(f_n)
    f1_mad = compute_median_absolute_deviation(f_n)
    f1_rms = compute_root_mean_square(f_n)
    f1_iqr = compute_interquartile_range(f_n)
    f1_per75 = compute_percentile_75(f_n)
    f1_kurtosis = compute_kurtosis(f_n)
    f1_min_max = compute_min_max(f_n)
    f1_sma = compute_signal_magnitude_area(f_n)
    f1_zcr = compute_zero_crossing_rate(f_n)
    f1_sc = compute_spectral_centroid(f_n)
    f1_entropy = compute_spectral_entropy(f_n)
    f1_energy = compute_spectral_energy(f_n)
    f1_pfreq = compute_principle_frequency(f_n)
    return f1_mean, f1_var, f1_mad, f1_rms, f1_iqr, f1_per75, f1_kurtosis, f1_min_max, f1_sma, f1_zcr, f1_sc, f1_entropy, f1_energy, f1_pfreq

def extract_raw_data_features(X):
    new_features = []
#     rows = X.shape[0]
#     cols = X.shape[1]
#     for row in range(rows):
#         features = []
#         for col in range(cols):
#             f_n = np.array(X[row][col])
#             feature = extract_raw_data_features_per_row(f_n)
#             features.append(feature)
#         new_features.append(features)
    for col in ["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z"]:
        features = []
        f_n = np.array(X[col])
        feature = extract_raw_data_features_per_row(f_n)
#         features.append(feature)
        new_features.append(feature)
#     new
#     return pd.DataFrame(data=np.array(new_features), columns=["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z"] )
    return new_features

# cols = ["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z"]
# final_data = extract_raw_data_features(np.array(pdata[cols]))

In [66]:
def create_featureslist(final_data):
#     new_data = []
#     for row in range(0,np.array(final_data).shape[0]):
#         row_features = []
#         for col in range(0,np.array(final_data).shape[1]):
#             feature_list = np.array(final_data)[row][col]
#             for feature in feature_list:
#                 row_features.append(feature)
#         new_data.append(row_features)
#         print("Row: " + str(row) + "done")

#     feature_name_list = ["_mean", "_var", "_mad", "_rms", "_iqr", "_per75", "_kurtosis", "_min_max", "_sma", "_zcr", "_sc", "_entropy", "_energy", "_pfreq"]
#     cols = []
#     for things in ["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z"]:
#         for feature in feature_name_list:
#             cols.append(things+feature)

#     data = pd.DataFrame(data=np.array(new_data), columns=cols)
#     data = data.dropna()
    new_data = []
    for row in range(np.array(final_data).shape[0]):
        for col in range(np.array(final_data).shape[1]):
            new_data.append(final_data[row][col])
    return new_data
#     return(data)

In [67]:
def combine_rows(data):
    combined_raw = []
    new_data = []
#     new_data = pd.DataFrame(columns=['acceleration_x', 'acceleration_y', 'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z'])
    for col in range(6):
        for row in range(len(data)):
            combined_raw.append(data[row][col])
        new_data.append(combined_raw)
        combined_raw = []
    return new_data

In [68]:
# %pip install tensorflow

In [127]:
#define a function which accepts raw data parameter.
#raw data would be a queue the form 600 * ['W',acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z]
# raw_data = []
# pkt1 = ('W',0.2,0.2,0.3,0.4,0.5,0.6)
# pkt2 = ('W',0.1,0.1,0.1,1.1,1.2,0.1)
# pkt3 = ('W',1.1,1.2,1.3,0.1,100000,1.6)
# raw_data.append(pkt1)
# raw_data.append(pkt2)
# raw_data.append(pkt3)
raw_data = [['W', -79.35, -3.76, -51.5, -8.0, -354.0, 5161.0],
 ['W', -78.67, -3.98, -51.26, 465.0, -164.0, 5026.0],
 ['W', -77.06, -4.48, -51.55, -10.0, 60.0, 5065.0],
 ['W', -76.73, -4.56, -50.97, 83.0, 74.0, 5282.0],
 ['W', -76.97, -4.15, -50.37, 175.0, -182.0, 5146.0],
 ['W', -77.92, -3.71, -49.08, 204.0, -289.0, 5080.0],
 ['W', -77.02, -3.73, -48.65, 315.0, -451.0, 5207.0],
 ['W', -77.42, -3.93, -48.47, 261.0, -634.0, 5192.0],
 ['W', -78.37, -5.06, -48.75, 73.0, -142.0, 5054.0],
 ['W', -76.93, -6.18, -49.51, 58.0, 420.0, 5104.0]]

def process(raw_data):
    data = pd.DataFrame(columns=['acceleration_x', 'acceleration_y', 'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z'])
    for i in range(len(raw_data)):
        list_row = [raw_data[i][1],raw_data[i][2],raw_data[i][3],raw_data[i][4],raw_data[i][5],raw_data[i][6]]
        data.loc[len(data)] = list_row
    final_data = combine_rows(data.values.tolist())
    final_data = pd.DataFrame(data=np.array(final_data).T, columns = ["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z"])
    final_data = extract_raw_data_features(final_data)
    final_data = np.array([create_featureslist(final_data)])
#     final_data = np.swapaxes(final_data, 0, 1)
#     final_data = list(map(lambda x: x[0], final_data))
#     print(len(final_data[0]))
#     scaler = StandardScaler()
#     scaler.fit(np.array(final_data).reshape(1, -1).tolist())
#     final_data = scaler.transform(np.array(final_data).reshape(1, -1))
#     print(final_data)
#     final_data = pd.DataFrame(final_data)
#     final_data = final_data[final_data.columns[:84]].astype('float64').to_numpy()
    print(final_data.tolist())
    scaler=load('./hardware_ai/cg4002/scaler.joblib')
#     print(scaler)
#     scaler = StandardScaler()
    final_data = scaler.transform(final_data.tolist())
    print(final_data.tolist())

#     final_data = scaler.transform(final_data.tolist())
#     print(final_data.tolist())

    model = load_model('./hardware_ai/cg4002/detection.h5')
#     model.get_weights()
#     final_data = np.array([final_data])
    #generate prediction from model
    output = model.predict(final_data).tolist()[0]
    print(output)
    prediction = output.index(max(output))

    if prediction == 0:
        return "reload"
    if prediction == 1:
        return "shield"
    if prediction == 2:
        return "grenade"
    if prediction == 3:
        return "logout"
    if prediction == 4:
        return ""
#     return prediction
    
#     return output
# def process(raw_data):
#     print("Hello. I have received the raw data." + "This is the first packed." + raw_data[0])
#     return 1

process(raw_data)      
    
#return lower case string // if not an action, return none.

/usr/lib/python3/dist-packages/scipy/signal/spectral.py:1967: UserWarning: nperseg = 256 is greater than input length  = 10, using nperseg = 10
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/share/pynq-venv/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[[-77.644, 0.7146439999999978, 0.6078660000000055, 77.64860191915885, 1.2750000000000057, -76.9825, -0.7880131133299799, 2.6199999999999903, -77.644, 0.0, 0.008772247276505881, 1.5754229799853303, 2.625485902055415, 0.30000000000000004, -4.354, 0.5409239999999998, 0.5114970000000006, 4.41568114790912, 0.7375000000000003, -3.8024999999999998, 1.0566189179612984, 2.4699999999999998, -4.354, 0.0, 0.10516228726399005, 1.0655058176266776, 2.519575499848079, 0.1, -50.010999999999996, 1.4129089999999995, 1.8384239999999976, 50.025123987852346, 2.355000000000004, -48.832499999999996, -1.685851221190259, 3.0799999999999983, -50.010999999999996, 0.0, 0.00929485540636791, 0.8465391852224279, 75.31602533337313, 0.1, 161.6, 21041.239999999998, 153.4491, 217.15386250306486, 185.0, 246.75, -0.5069505694633762, 475.0, 161.6, 3.0, 0.3781375466327289, 1.23514136871379, 2763000215.1479387, 0.1, -166.2, 80362.96, 306.8982, 328.61132055971535, 347.25, 9.5, -0.2133283357660769, 1054.0, -166.2, 3.0, 0.374268

''

In [ ]:
import pandas as pd
def load_file(filepath):
  dataframe = pd.read_csv(filepath)
  return dataframe

data = load_file("./hardware_ai/cg4002/data.csv")
data.head()
data = data.dropna()
data

In [ ]:
x = data[data.columns[:84]].astype('float64').to_numpy()
x[0]

In [ ]:
y = data["prediction"].to_numpy()
x = data[data.columns[1:85]].to_numpy()

In [ ]:
x = np.array([1] * 84)
y = np.array([0])
x

In [ ]:
correct_pred = 0
#Setting a threshold for onset detection
# THRESHOLD = 0.01
# threshold_sum = 0
dma = overlay.axi_dma_0        
# take in 10 datapoints, average out, compare to threshold
in_buffer = allocate(shape=x.shape, dtype=np.double)
out_buffer = allocate(shape=(1,), dtype=np.double)
print(in_buffer)
print(out_buffer)
for i in range(x.shape[0]):
    overlay = Overlay("./hardware_ai/cg4002/design_1_wrapper.bit")
    dma = overlay.axi_dma_0        
#     inputs = x[n]
#     for i in range(84):
    in_buffer[i] = x[i];
#         threshold_sum += inputs[i];
#         print(threshold_sum)
# print(in_buffer)
# print(out_buffer)
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
# print("aaa1")
# print(in_buffer)
# print(out_buffer)
dma.sendchannel.wait()
# print("aaa2")
dma.recvchannel.wait()
# print("aaa3")

out_buffer
# if y[i] == np.argmax(out_buffer):
#     correct_pred += 1;

# accuracy = 100 * correct_pred / len(x)
# print('Accuracy: {}'. format(accuracy))
# print('Process time: ' + str(time.time() - start) + "s")
